In [1]:
import itertools
from itertools import groupby
import glob
import hashlib

# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

import pandas as pd

# Simhash

Simhash es una técnica para estimar rápidamente cuán similares son dos conjuntos. Es usada por <strong>Crawler</strong> de Google para encontrar páginas duplicadas o similares.

El algoritmo de Simhash consta de los siguientes 4 pasos:

<ol>
    <li>
        Procesar el documento dejando un conjunto de variables con un peso asociado. En este caso se considerarán estas variable como <i>palabras</i> con su frecuencia como peso.
    </li>
    <li>
        Generar un valor único de hash para cada variable de <i>b</i> bits (el valor deseado de la huella difital (<i>Fingerprint</i>). 
    </li>
    <li>
        Crear un <i>b-dimensional</i> vector $V$ en el que se sumará o restará en cada posición el peso de cada variable dependiendo de si el bit de su hash en esa misma posición es 1 o 0, respectivamente.
    </li>
    <li>
        Después de que todas las variables hayan sido procesadas, se genera una huella digital de $b$ bits poniendo los bits a 1 o 0 dependiendo de si el valor en la posición del vector $V$ es positiva o negativa, respectivamente.
    </li>
</ol>

In [2]:
data = "Tropical fish include fish found in tropical environments around the world, \
        including both freshwater and salt water species."

## Paso 1. Procesamiento del documento

El procesamiento del documento tendrá como primer paso la tokenización (segmentación) del documento para eliminar la palabras menos usadas (**stop words**) y los signos de puntuación y, en caso de que se desee, poner las palabras en minúscula para, finalmente, generar una lista con las palabras necesarias. Estas acciones son las que he querido realizar para este ejercicio, pero se podrían hacer otras dependiendo del idioma del documento, tipo de análisis que se desee realizar, etc.

In [3]:
stopWords = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

data_lower = data.lower()
words = tokenizer.tokenize(data_lower)
words = list(filter(lambda token: token not in stopwords.words('english'), words))
 
print(words)

['tropical', 'fish', 'include', 'fish', 'found', 'tropical', 'environments', 'around', 'world', 'including', 'freshwater', 'salt', 'water', 'species']


La segunda parte del procesamiento del documento será asignar un peso a cada palabra que, en este caso, es su frecuencia de aparación.

In [4]:
words_dict = {k:sum(1 for _ in g) for k, g in groupby(sorted(words))}
words_dict

{'around': 1,
 'environments': 1,
 'fish': 2,
 'found': 1,
 'freshwater': 1,
 'include': 1,
 'including': 1,
 'salt': 1,
 'species': 1,
 'tropical': 2,
 'water': 1,
 'world': 1}

## Paso 2 y 3. Generación de los valores hash y creación del vector $V$

La función de hash empleada en este caso es **MD5**, la cual genera una secuencia de 128 bits. Para poder coger los últimos $b$ bits de cada secuencia, se ha utilizado **& ((1 << b) - 1)**, lo que crea una máscara de $b$ bits igual a 1.

Posteriormente, se genera el hash por cada palabra y se suma o resta en la posición correspondiente de vector $V$ el peso de la palabra dependiendo de si el bit de su hash en esa misma posición es 1 o 0, respectivamente.

In [5]:
def hashfunc(x, b=128):
    hexad = int(hashlib.md5(x).hexdigest(), 16)
    return hexad & ((1 << b) - 1)

b = 8

print('Valores de hash:\n')

v = [0] * b
for word in words_dict.items():
    h = hashfunc(word[0].encode('utf-8'), b)
    w = word[1]
    
    print(word[0], ' - ', bin(h))
    
    for i in range(b):
        v[b - 1 - i] += w if h >> i & 1 else -w

print('\n\nVector V:', v)

Valores de hash:

environments  -  0b101111
salt  -  0b11101110
species  -  0b10110010
world  -  0b11100111
fish  -  0b1001
include  -  0b10000001
around  -  0b10010101
freshwater  -  0b11000011
tropical  -  0b11110101
found  -  0b1111010
including  -  0b11000110
water  -  0b101100


Vector V: [4, 0, 2, -4, -2, 2, 0, 4]


## Paso 4. Generación de la huella digital

Por último, se genera la huella digital (*Fingerprint*) de $b$ bits poniendo los bits a 1 o 0 dependiendo de si el valor en la posición del vector $V$ es positiva o negativa, respectivamente.

In [6]:
fingerprint = 0
for i in range(b):
    if v[b - 1 - i] > 0:
        fingerprint |= (1 << i)

print('Fingerprint:', fingerprint)
print('Fingerprint en bits:', bin(fingerprint))

Fingerprint: 165
Fingerprint en bits: 0b10100101


# Código completo

Función de tokenización en la que se da al usuario la elección de poner todas las palabras de un documento en minúscula.

In [7]:
def tokenize(text, lower=True, lang='english'):
    stopWords = set(stopwords.words(lang))
    tokenizer = RegexpTokenizer(r'\w+')
    
    if lower:
        text = text.lower()
    words = tokenizer.tokenize(text)
    words = list(filter(lambda token: token not in stopwords.words('english'), words))
    
    return words

Función hash explicada anteriormente.

In [8]:
def hashfunc(x, b=128):
    hexad = int(hashlib.md5(x).hexdigest(), 16)
    return hexad & ((1 << b) - 1)

Algoritmo de **Simhash** en el que se pasa por parámentro un documento tokenizado y, opcionalmente, un valor para $b$.

In [9]:
def simhash(tokenized_doc, b=128):    
    words_dict = {k:sum(1 for _ in g) for k, g in groupby(sorted(tokenized_doc))}

    v = [0] * b
    for word in words_dict.items():
        h = hashfunc(word[0].encode('utf-8'), b)
        w = word[1]

        for i in range(b):
            v[b - 1 - i] += w if h >> i & 1 else -w
            
    fingerprint = 0
    for i in range(b):
        if v[b - 1 - i] > 0:
            fingerprint |= (1 << i)
    
    return fingerprint

Función con la que se puede calcular la distancia (número de bits diferentes) entre dos huellas digitales. El símbolo **^** es el operador *OR exclusivo bit a bit*, en el que entre dos secuencias de bits si un bit es 0 y el otro bit es 1, el bit del resultado correspondiente se establece en 1. De lo contrario, el bit del resultado correspondiente se establece en 0.

In [10]:
def distance(fingerprint1, fingerprint2, b=128):
    x = (fingerprint1 ^ fingerprint2) & ((1 << b) - 1)
    dist = 0
    while x:
        dist += 1
        x &= x - 1
        
    return dist

Por último, muestro un ejemplo de Simhash de 8 bits de dos documentos iguales, donde la única diferencia es que en el tokenizado del segundo se ha especificado que no se convierta a minúscula ninguna palabra.

In [11]:
doc1 = "Tropical fish include fish found in tropical environments around the world, \
        including both freshwater and salt water species."

doc2 = "Tropical fish include fish found in tropical environments around the world, \
        including both freshwater and salt water species."

b = 8

# Simhash of doc1
tokenized_doc1 = tokenize(doc1)
fingerprint1 = simhash(tokenized_doc1, b)

# Simhash of doc2
tokenized_doc2 = tokenize(doc2, lower=False)
fingerprint2 = simhash(tokenized_doc2, b)

# Distance between doc1 and doc2
dist = distance(fingerprint1, fingerprint2, b)

print('Fingerprint del primer documento', fingerprint1)
print('Fingerprint del primer documento', fingerprint2)
print('Bits diferentes entre ambos documentos:', dist, 'de', b)

Fingerprint del primer documento 165
Fingerprint del primer documento 167
Bits diferentes entre ambos documentos: 1 de 8


# Ejemplo con libros

Se van a comparar los siguientes libros en inglés:

<ol>
    <li>Alicia en el País de las Maravillas</li>
    <li>Drácula</li>
    <li>La primera mitad del mismo libro de Drácula. Lo he llamado **dracula_break**</li>
    <li>Frankestein</li>
    <li>Moby Dick</li>
</ol>

Vamos a utilizar la siguiente función para generar una matriz de las distancias entre cada uno de los libros.

In [12]:
def distances_matrix(simhashes):
    iterables = [simhashes, simhashes]
    simhashes_comb = {key : [] for key in simhashes}

    for t in itertools.product(*iterables):
        dist = distance(simhashes[t[0]], simhashes[t[1]], b)
        simhashes_comb[t[0]].append(dist)

    df_matrix = pd.DataFrame.from_items(simhashes_comb.items(), 
                                        orient='index', 
                                        columns=list(simhashes_comb.keys()))
    return df_matrix

### Con 8 bits

In [13]:
b = 8
simhashes_8 = {}

for files in glob.glob("*.txt"):
    infile = open(files, mode='r')
    filename = infile.name
    a = infile.read()
    infile.close()
    
    token = tokenize(a)
    simh = simhash(token, b)
    simhashes_8[filename] = simh
        
simhashes_8

{'alice_adventures.txt': 92,
 'dracula.txt': 90,
 'dracula_break.txt': 90,
 'frankestein.txt': 122,
 'moby_dick.txt': 74}

In [14]:
distances_matrix(simhashes_8)

,frankestein.txt,dracula_break.txt,moby_dick.txt,dracula.txt,alice_adventures.txt
frankestein.txt,0,2,1,1,3
dracula_break.txt,1,1,0,0,2
moby_dick.txt,2,0,1,1,3
dracula.txt,1,1,0,0,2
alice_adventures.txt,3,3,2,2,0


### Con 64 bits

In [15]:
b = 64
simhashes_64 = {}

for files in glob.glob("*.txt"):
    infile = open(files, mode='r')
    filename = infile.name
    a = infile.read()
    infile.close()
    
    token = tokenize(a)
    simh = simhash(token, b)
    simhashes_64[filename] = simh
        
simhashes_64

{'alice_adventures.txt': 9984005856265639004,
 'dracula.txt': 10956864711739610202,
 'dracula_break.txt': 11100979968513971290,
 'frankestein.txt': 12271427276163265658,
 'moby_dick.txt': 724754358419373130}

In [16]:
distances_matrix(simhashes_64)

,frankestein.txt,dracula_break.txt,moby_dick.txt,dracula.txt,alice_adventures.txt
frankestein.txt,0,16,13,15,22
dracula_break.txt,13,11,0,4,15
moby_dick.txt,16,0,11,13,16
dracula.txt,15,13,4,0,17
alice_adventures.txt,22,16,15,17,0


### Con 128 bits

In [17]:
b = 128
simhashes_128 = {}

for files in glob.glob("*.txt"):
    infile = open(files, mode='r')
    filename = infile.name
    a = infile.read()
    infile.close()
    
    token = tokenize(a)
    simh = simhash(token, b)
    simhashes_128[filename] = simh
        
simhashes_128

{'alice_adventures.txt': 327763483949349671204758320733936030812,
 'dracula.txt': 336108638123220734845428257125189063770,
 'dracula_break.txt': 336119022716937653385101981545975026778,
 'frankestein.txt': 316086574081351765772503192164555043962,
 'moby_dick.txt': 250884428747321189650351009777993169994}

In [18]:
distances_matrix(simhashes_128)

,frankestein.txt,dracula_break.txt,moby_dick.txt,dracula.txt,alice_adventures.txt
frankestein.txt,0,39,27,29,47
dracula_break.txt,27,28,0,6,34
moby_dick.txt,39,0,28,32,42
dracula.txt,29,32,6,0,36
alice_adventures.txt,47,42,34,36,0


### Resultados finales

Cuanto mayor es el tamaño de $b$, más se pueden apreciar las diferencias entre los libros a través de su distancia. Por ejemplo, entre el libro de Drácula entero y el de solo la primera mitad, con 8 bits nos muestra que no hay diferencias, mientras que con 64 y 128 ya podemos ver como sí que hay una pequeña diferencia. De todas maneras, con las 3 formas sabríamos que son documentos con un alto índice de duplicidad.

Otro ejemplo es el de Moby Dick. Con 8 y 64 bits es difícil ver con cuál tiene más diferencias. No obstante, con 128 bits se ve claramente como tiene una mayor diferencia con Alicia en el País de las Maravillas y Frankestein.